In [5]:
import os 
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
from langchain_groq import ChatGroq
groq_api = os.getenv("GROQ_API")

llm = ChatGroq(api_key=groq_api,model="llama-3.1-8b-instant")


In [21]:
from langchain.schema import(AIMessage,HumanMessage,SystemMessage)

speech = """]The demise of former Prime Minister Dr. Manmohan Singh ji has deeply pained our hearts. His passing is a tremendous loss for us as a nation. Coming to Bharat during the time of partition after losing so much, and achieving remarkable success in every field of life, is no ordinary feat. His life serves as a lesson for future generations on how to rise above hardships and challenges to reach great heights.

He will always be remembered as a kind-hearted individual, a scholarly economist, and a leader dedicated to reforms. As an economist, he served the Government of Bharat in various capacities. During a challenging time, he played the role of the Governor of the Reserve Bank of India. As the Finance Minister in the government of former Prime Minister and Bharat Ratna Shri P.V. Narasimha Rao ji, he steered the country out of a financial crisis and paved the way for a new economic direction. His contributions as the Prime Minister towards the country’s development and progress will always be cherished.

His commitment to the people and the nation’s development will forever be held in high regard. Dr. Manmohan Singh ji’s life was a reflection of honesty and simplicity. He was an extraordinary parliamentarian. His humility, gentleness, and intellect defined his parliamentary life. I remember mentioning earlier this year, when his tenure in the Rajya Sabha ended, that his dedication as a Member of Parliament is an inspiration to all. Even during crucial moments of parliamentary sessions, he would attend in a wheelchair and fulfil his parliamentary duties.

Despite being educated at some of the world’s most prestigious institutions and holding numerous top positions in the government, he never forgot the values of his humble background. Rising above partisan politics, he always maintained connections with people across party lines and remained approachable to everyone. During my tenure as Chief Minister, I had open discussions with Dr. Manmohan Singh ji on various national and international issues. Even after coming to Delhi, I would frequently meet and converse with him. I will always remember our discussions about the country and our meetings. Recently, I spoke to him on his birthday."""

In [8]:
llm.get_num_tokens(speech)

c:\Users\AKM\Documents\GenrativeAI\Differen_applications\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


448

In [9]:
chat_message = [
SystemMessage("you are an expert with experties of summarizing speeched"),
HumanMessage(f"please provide me the summary of {speech} it should be short and consize ")
]

In [10]:
## getting the summary very easy and first technique but will not work with large data
llm.invoke(chat_message)

AIMessage(content="Here's a concise summary:\n\nFormer Prime Minister Dr. Manmohan Singh's passing has caused immense pain and is a significant loss for the nation. Despite facing hardships, he achieved remarkable success in various fields. He will be remembered as a kind-hearted, scholarly economist, and dedicated leader who worked tirelessly for reforms. As the Governor of the Reserve Bank of India, Finance Minister, and Prime Minister, he steered the country through financial crises and paved the way for a new economic direction. His commitment to the nation's development, honesty, and simplicity will be cherished forever.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 116, 'prompt_tokens': 490, 'total_tokens': 606, 'completion_time': 0.154666667, 'prompt_time': 0.0332881, 'queue_time': 0.020590948999999997, 'total_time': 0.187954767}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_9cb648b966', 'finish_reason': 'stop', 'logprobs': No

## second soln => prompt template summarization

In [32]:
from langchain.chains import LLMChain ## whenever we combine prompt tamplate with llm i.e is llmchain
from langchain_core.prompts import PromptTemplate

generic_prompt = """

write a summary of the following speech:
speech:{speech}
Translate the precise summary to {langauge}
"""
prompt = PromptTemplate(
    input_variables=["speech","langauge"],
    template = generic_prompt
)
prompt

PromptTemplate(input_variables=['langauge', 'speech'], input_types={}, partial_variables={}, template='\n\nwrite a summary of the following speech:\nspeech:{speech}\nTranslate the precise summary to {langauge}\n')

In [28]:
chain = LLMChain(llm=llm,prompt=prompt)
summary = chain.invoke({"speech":speech,"langauge":"Marathi"})


## StuffDoc chain text summarization

In [33]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("AdityaMoreINT354 (2).pdf")
document = loader.load_and_split()

In [34]:
document

[Document(metadata={'source': 'AdityaMoreINT354 (2).pdf', 'page': 0}, page_content='Sentiment Analysis for Effective Stock Market Prediction \nAditya More \n \n1Department of Computer Science and Engineering, Lovely Professional University,   \nPunjab, India \n \n \nAbstract: We analyze the impact of sentiment and attention variables on the stock market volatility by using a \nnovel and extensive dataset that combines social media, news articles, information consumption, and search engine \ndata. We apply a state-of-the art sentiment classification technique in order to investigate the question of whether \nsentiment and attention measures contain additional predictive power for realized volatility when controlling for a \nwide range of economic and financial predictors. Using a penalized regression framework, we identify th e most \nrelevant variables to be investors’ attention, as measured by the number of Google searches on financial keywords \n(e.g. ‘‘financial market’’ and ‘‘stock

In [35]:
template = """

Write a concise and short summary of the following speech,
speech:{text}
"""
prompt = PromptTemplate(input_variables=["text"],template=template)

In [43]:
from langchain.chains.summarize import load_summarize_chain

chain = load_summarize_chain(llm,chain_type="stuff",prompt=prompt)

In [44]:
output = chain.run(document)

In [45]:
output

'Here\'s a concise and short summary of the speech "Sentiment Analysis for Effective Stock Market Prediction" by Aditya More:\n\nThe speech discusses a novel approach to stock market prediction using sentiment analysis and machine learning algorithms. The approach combines social media, news articles, and search engine data to analyze sentiment and attention variables that impact stock market volatility. The study uses a dataset sourced from Kaggle and applies various machine learning algorithms, including random forest, decision tree, SVM, Xgboost, voting ensemble, and Naïve Bayes. The results show that attention and sentiment variables can improve volatility forecasts, although the improvements are relatively small. The study concludes that combining sentiment polarity with Sensex points provides an efficient result for stock market forecasters, allowing them to determine when to buy or sell stocks.'

## map reduce 


In [46]:
from  langchain_text_splitters import RecursiveCharacterTextSplitter
split = RecursiveCharacterTextSplitter(chunk_size=400,chunk_overlap=100)
doc = split.split_documents(document)

In [50]:
chunk_prompt = """
please summarize the bello text:
text: {text}
summary:
"""

map_prompt_temp = PromptTemplate(input_variables=["text"],template=chunk_prompt)

In [48]:
final_prompt = "combine all the chunks and give me the summary. it shout be short and consie here is the text:{text}"
map_final_template = PromptTemplate(input_variables=["text"],template=final_prompt)

In [51]:
summa = load_summarize_chain(llm,chain_type="map_reduce",map_prompt=map_prompt_temp,combine_prompt=map_final_template)

In [52]:
summa.run(doc)

Token indices sequence length is longer than the specified maximum sequence length for this model (3079 > 1024). Running this sequence through the model will result in indexing errors


'Here\'s a short and concise summary:\n\nThe journal "The Journal of Finance and Data Science" focuses on the intersection of finance and data science, publishing research on topics like stock market prediction, financial modeling, and machine learning. A research paper titled "Sentiment Analysis for Effective Stock Market Prediction" demonstrates the potential of using sentiment analysis and machine learning algorithms, such as RandomForest and SVM, to improve stock market forecasts, but notes that improvements are relatively small from an economic perspective.'